# Mount Google Drive
In this example, the data is saved in the folder of personal google drive.

First you have to upload the data to your google drive, then connect the drive.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Python Packages
Although most of the commonly used Python libraries are pre-installed, new libraries can be installed using the below packages:

!pip install [package name]

In [ ]:
!pip install tsfresh
# tsfresh is a python package, which automatically calculates a large number of time series characteristics (features)

# Import Packages

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
sns.set()
import matplotlib.pyplot as plt
plt.rcParams['ytick.labelsize'] = "x-large"
plt.rcParams['xtick.labelsize'] = "x-large"
plt.rcParams['axes.labelsize'] = "x-large"
plt.rcParams['figure.titlesize'] = "x-large"


from sklearn.preprocessing import normalize
import lightgbm as lgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV

from sklearn.metrics import mean_squared_error

import keras

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import TimeDistributed

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

# Data Description
C-MAPSS data set which contains turbofan engine degradation data is a widely used prognostic benchmark data for predicting the Remaining useful life (RUL). This data set is simulated by the tool Commercial Modular Aero Propulsion System Simulation (C-MAPSS) developed by NASA. Run to failure simulations were performed for engines with varying degrees of initial wear but in a healthy state. During each cycle in the simulation, one sample of all 21 sensors such as physical core speed, temperature at fan inlet and pressure at fan inlet etc will be recorded once. As the simulation progresses, the performance of the turbofan engine degrades until it loses functionality. 

C-MAPSS data consists of four sub-data sets with different operational conditions and fault patterns. 

|         Dataset        | FD001 | FD002 | FD003 | FD004 |
|:----------------------:|:-----:|:-----:|:-----:|:-----:|
|      Training set      |  100  |  260  |  100  |  249  |
|        Test set        |  100  |  259  |  100  |  248  |
| Operational conditions |   1   |   6   |   1   |   6   |
| Fault conditions       | 1     | 1     | 2     | 2     |


As shown Table above, each sub-data set has been split into a training set and a test set. The training sets contain sensor records for all cycles in the run to failure simulation. Unlike the training sets, the test sets only contain partial temporal sensor records which stopped at a time prior to the failure. The task is to predict the RUL of each engine in the test sets by using the training sets with the given sensor records. The corresponding RUL to test sets has been provided. With this, the performance of the model can be verified. 

The data provieded as text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:

unit number

time, in cycles

operational setting 1

operational setting 2

operational setting 3

sensor measurement 1

sensor measurement 2 

sensor measurement 3 

...

sensor measurement 26

# Data Exploration and Preparation
take FD001 as example

In [ ]:
# Load the Data
Path_to_data = "drive/My Drive/PSDA2020/"

column_name = ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
               's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
               's15', 's16', 's17', 's18', 's19', 's20', 's21']
# training data set
train_FD001 = pd.read_table(Path_to_data+"train_FD001.txt", header=None, delim_whitespace=True)
train_FD001.columns = column_name

# test data set
test_FD001 = pd.read_table(Path_to_data+"test_FD001.txt", header=None, delim_whitespace=True)
test_FD001.columns = column_name

# RUL for test data set
RUL_FD001 = pd.read_table(Path_to_data+"RUL_FD001.txt", header=None, delim_whitespace=True)

train_FD001.head()

In this sub dataset we have **100** engines (engine_id) which are monitored over time (cycle). Each engine had operational_settings and sensor_measurements recorded for each cycle. The RUL is the amount of cycles an engine has left before it needs maintenance. What makes this data set special is that the engines run all the way until failure, giving us precise RUL information for every engine at every point in time.

In [ ]:
def add_RUL(col):
    # Reverse the cycle evolution, where remaining time of a machine is 0 at the failure.
    # It is assumed here that the state of the machine is linearly deteriorating
    return col[::-1]-1
# Calculate RUL for each time point of each engine  
train_FD001['rul'] = train_FD001[['engine_id', 'cycle']].groupby('engine_id').transform(add_RUL)

**Is there any other way to define target lable (RUL) ?** 

In [ ]:
#Plotting an engine's classic linear degradation function
engine_id = 1
df_of_id = train_FD001[train_FD001['engine_id']==engine_id]
plt.plot(df_of_id.rul, label="RUL")
#plt.plot(df_of_id.Cycle, label="Cycle")
plt.legend()
plt.title("Classic linear degrading RUL of engine {}".format(engine_id))
plt.show()

A second way to define the target label/Remaining useful lifetime (RUL) is called piece-wise linear degradation function. The method above is a classic linear degradation function.
The piece-wise degradation function assumes that the degradation of the RUL starts at a specific point in lifetime. Before that point the function is constant, after that point it starts degrading linearly. This point of degradation can be seen in graphs below when the curves of specific sensor's values start to increase or decrease steadily over time.

Piece-wise linear degradation function. Warning before running cell: overrides linear degradation function above!

In [ ]:
# piece-wise linear degradation function
id='engine_id'
MAXLIFE = 120 # or 125 , 130
rul = [] 
for _id in set(train_FD001[id]):
    train_FD001_id =  train_FD001[train_FD001[id] == _id]
    cycle_list = train_FD001_id['cycle'].tolist()
    max_cycle = max(cycle_list)

    breaking_point = max_cycle - MAXLIFE
    kink_RUL = []
    for i in range(0, len(cycle_list)):
        if i < breaking_point:
            kink_RUL.append(MAXLIFE)
        else:
            tmp = max_cycle-i-1
            kink_RUL.append(tmp)
    rul.extend(kink_RUL)

train_FD001['rul'] = rul

Plotting piece-wise linear degradation function.

In [ ]:
#Plotting an engine's piece-wise linear degradation function
engine_id = 1
df_of_id = train_FD001[train_FD001['engine_id']==engine_id]
plt.plot(df_of_id.rul, label="RUL")
#plt.plot(df_of_id.Cycle, label="Cycle")
plt.legend()
plt.title("piece-wise linear degrading RUL of engine {}".format(engine_id))
plt.show()

In [ ]:
# Visualize the RUL curve of some engines (1,2,3,4,5,6)
g = sns.PairGrid(data=train_FD001.reset_index().query('engine_id < 7') ,
                 x_vars=["index"],
                 y_vars=['rul'],
                 hue="engine_id", height=3, aspect=2.5)

g = g.map(plt.plot, alpha=1)
g = g.add_legend()

In [ ]:
# Visualize some sensor curves of some engines 
g = sns.PairGrid(data=train_FD001.query('engine_id < 5') ,
                 x_vars=["rul"],
                 y_vars=['s1','s2'],
                 hue="engine_id", height=3, aspect=2.5)

g = g.map(plt.plot, alpha=1)
g = g.add_legend()

# As shown in the figure, some sensors are not related to RUL. 
# The values of some sensors change with the state of the machine. 
# Visualization can help filter features

In [ ]:
# Distribution of maximum life cycle
train_FD001[['engine_id', 'rul']].groupby('engine_id').apply(np.max)["rul"].hist(bins=20)
plt.xlabel("max life cycle")
plt.ylabel("Count")
plt.show()

**Can you do more visualization？ Please give a simple summary or explanation for each visualization**

In [ ]:
# The following graphs show the change in value of every sensor over time (here for engine number one). X-axis: cycle number, Y-axis: sensor values
engine_id = 1
plt.figure(figsize=(15,2.5*21))
Dataframe_id = train_FD001[train_FD001["engine_id"]==engine_id]
for i in range(21):
    a = plt.subplot(26,1,i+1)
    a.plot(Dataframe_id.index.values,Dataframe_id.iloc[:,i+5].values)
    a.title.set_text("Sensor " + str(i+1) + ", column "+ str(i+6))
    plt.tight_layout()
plt.show()



It can be seen that most sensors either tend to go up or tend to go down over time. That could be caused by the change of the engine's state. Other sensors (1, 5, 6, 10, 16, 18, 19) have constant values over time. 

In [ ]:
# Boxplots of sensor data.
plt.figure(figsize = (16, 21))

for i in range(21):
    temp_data = train_FD001.iloc[:,i+5]
    plt.subplot(7,3,i+1)
    plt.boxplot(temp_data)
    plt.title("Sensor " + str(i+1) + ", column "+ str(i+6))
plt.show()

It can be seen, that sensors 1, 5, 6, 10, 16, 18 and 19 have constant values over time. That means that these sensors seem to have no further impact on the lifespan of an engine. It is indicted that only sensors 2, 3, 4, 7, 8, 9, 11, 12 ,13 ,14 ,15 ,17, 20, 21 need to be considered in further calculations/visualizations.

In [ ]:
sensor_names = column_name[5:]
train_FD001[sensor_names].describe().transpose()

The standard deviation of sensors 18 and 19 clearly show that they are obsolete for RUL since they hold no useful information. Sensors 1, 5, 10 and 16 only have a very small standard deviation.

In [ ]:
# prepare the data and normalization
train_y = train_FD001['rul']
features = train_FD001.columns.drop(['engine_id', 'cycle', 'rul'])
train_x = train_FD001[features]
test_x = test_FD001[features]


# z score normalization
mean = train_x.mean()
std = train_x.std()
std.replace(0, 1, inplace=True)

train_x = (train_x - mean) / std
test_x = (test_x - mean) / std

from sklearn.utils import shuffle
x, y = shuffle(train_x, train_y)
train_FD001.head()



**Here only the values at each time point (cycle) are used to predict the RUL. Temporal relationship is ignored. How to use Temporal relationship?**

tip : Sliding Window


Sliding Window can be used to capture temporal relationships in a dataset. A window is pushed over the data whih are then combined. Window size and stride are important parameters. For an example, see LSTM model below.

# Modeling

In [ ]:
# Random Forest with default Hyper parameters
rf_model = RandomForestRegressor()
rf_model.fit(x,y)
rf_prediction = rf_model.predict(train_x)
plt.plot(rf_prediction[:500])
plt.plot(train_FD001["rul"][:500])
plt.show()

In [ ]:
# Lasso model with default Hyper parameters
ls_model = LassoCV()
ls_model.fit(x,y)
ls_prediction = ls_model.predict(train_x)
plt.plot(ls_prediction[:500])
plt.plot(train_FD001["rul"][:500])
plt.show()

**How to tune hyperparameters and select models?** 

**Neural network model?**

In Predictive Maintenance regression models are used to predict Remaining Useful Lifetime (RUL).
Model selection: What kind of output should the model give? Availability of sufficient historical data? Here: Classification problem of time-series data! Therefore (Deep) CNNs and LSTMs should be chosen. As results indicate/prove.
Hyperparameter tuning: Specify which (hyper-)parameters are to tune. First, start with more or less random values. Then opitmizer like BOHB can be used. BOHB consists of the Bayesian optimization (BO) and the Hyperband (HB).

In [ ]:
# LGBMRegressor with default Hyper parameters
reg_model = lgb.LGBMRegressor(random_state=12)
reg_model.fit(x, y)
reg_prediction = reg_model.predict(train_x)
#plt.figure(figsize=(12,5))
plt.plot((reg_prediction[:2000]), label="Prediction")
plt.plot(train_FD001["rul"][:2000])
plt.legend()
plt.show()

# Evaluation

In [ ]:

# Since only the value at one time point is used, it can be seen that a lot of data in the test set is not used

test_x['engine_id'] = test_FD001['engine_id']
test_input = []
for id in test_x['engine_id'].unique():
  
  test_input.append(test_x[test_x['engine_id']==id].iloc[-1,:-1].values)

test_input = np.array(test_input)
test_input

In [ ]:
# Random forest

rf_test_prediction = rf_model.predict(test_input)

rf_rmse = np.sqrt(mean_squared_error(rf_test_prediction, RUL_FD001.values.reshape(-1)))

print("The RMSE of random forest on test dataset FD001 is ",rf_rmse)

In [ ]:
# Lasso model

ls_test_prediction = ls_model.predict(test_input)

ls_rmse = np.sqrt(mean_squared_error(ls_test_prediction, RUL_FD001.values.reshape(-1)))

print("The RMSE of Lasso model on test dataset FD001 is ",ls_rmse)

In [ ]:
# LGBMRegressor model

reg_test_prediction = reg_model.predict(test_input)

reg_rmse = np.sqrt(mean_squared_error(reg_test_prediction, RUL_FD001.values.reshape(-1)))

print("The RMSE of LGBMRegressor model on test dataset FD001 is ",reg_rmse)

**What is your best result? If the used model is interpretable, what other conclusions can be summarized**

Here we use LSTM with sliding window and reach a better solution than Random Forest model, Lasso model and LGBMRegression model.

In [ ]:
# read training data 
train_df = pd.read_csv("drive/My Drive/PSDA2020/train_FD001.txt", sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [ ]:
# read test data
test_df = pd.read_csv("drive/My Drive/PSDA2020/test_FD001.txt", sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [ ]:
# read ground truth data
truth_df = pd.read_csv("drive/My Drive/PSDA2020/RUL_FD001.txt", sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [ ]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

In [ ]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

In [ ]:
# generate label columns for training data
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
train_df.head()

In [ ]:
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

In [ ]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

In [ ]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [ ]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df.head()

In [ ]:
# generate label columns label1 and label2 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

In [ ]:
# pick a window size of 50 cycles
sequence_length = 50

In [ ]:
# preparing data for visualizations 
engine_id3 = test_df[test_df['id'] == 3]
engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]
cols1 = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']
engine_id3_50cycleWindow1 = engine_id3_50cycleWindow[cols1]
cols2 = ['s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
engine_id3_50cycleWindow2 = engine_id3_50cycleWindow[cols2]

In [ ]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [ ]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

In [ ]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

In [ ]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

In [ ]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [ ]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

In [ ]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
%%time
# fit the network
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

In [ ]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

In [ ]:
# make predictions and compute confusion matrix
y_pred = (model.predict(seq_array) > 0.5).astype("int32")
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

In [ ]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

In [ ]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [ ]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

In [ ]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

In [ ]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

In [ ]:
# make predictions and compute confusion matrix
y_pred_test = (model.predict(seq_array_test_last) > 0.5).astype("int32")
y_true_test = label_array_test_last
lstm_rmse = np.sqrt(mean_squared_error(y_pred_test, y_true_test))
print("The RMSE of LSTM model on test dataset FD001 is ",100*lstm_rmse)
#print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
#cm = confusion_matrix(y_true_test, y_pred_test)
#cm

**LSTM** on test data set with **sliding window** has an RMSE of **17.96**

In [ ]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )